In [3]:
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset
from azureml.core.experiment import Experiment
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [4]:
ws = Workspace.from_config()

In [5]:
ws.datasets

{'Iris': DatasetRegistration(id='1fe4a774-82c1-4c71-b205-375051bb4b2b', name='Iris', version=1, description='', tags={}), 'BankCustomer': DatasetRegistration(id='fc963daa-041c-49a8-9587-678e2e792e8e', name='BankCustomer', version=1, description='', tags={})}

In [6]:
iris_ds = Dataset.get_by_name(workspace=ws, name="Iris")

In [7]:
iris_df = iris_ds.to_pandas_dataframe()

In [35]:
iris_df.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Class
0,5.10,3.50,1.40,0.20,Iris-setosa
1,4.90,3.00,1.40,0.20,Iris-setosa
2,4.70,3.20,1.30,0.20,Iris-setosa
3,4.60,3.10,1.50,0.20,Iris-setosa
4,5.00,3.60,1.40,0.20,Iris-setosa


In [36]:
iris_df.columns

Index(['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Class'], dtype='object')

In [37]:
iris_df = iris_df.rename(columns={
    "Column1": "SepalLength",
    "Column2": "SepalWidth",
    "Column3": "PetalLength",
    "Column4": "PetalWidth",
    "Column5": "Class"
})

In [38]:
iris_df.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Class
0,5.10,3.50,1.40,0.20,Iris-setosa
1,4.90,3.00,1.40,0.20,Iris-setosa
2,4.70,3.20,1.30,0.20,Iris-setosa
3,4.60,3.10,1.50,0.20,Iris-setosa
4,5.00,3.60,1.40,0.20,Iris-setosa


In [39]:
X_train, X_test = train_test_split(iris_df, test_size=0.2, random_state=42)

In [40]:
automl_settings = {
    "iteration_timeout_minutes": 2,
    "experiment_timeout_minutes": 15,
    "enable_early_stopping": True,
    "primary_metric": 'AUC_weighted',
    "featurization": 'auto',
    "n_cross_validations": 5,
}

In [44]:
automl_config = AutoMLConfig(task='classification',
    debug_log='automl_errors.log',
    training_data=X_train,
    label_column_name="Class",
    **automl_settings)

In [45]:
experiment = Experiment(ws, "iris_experiment")

In [46]:
run = experiment.submit(automl_config, show_output=True)

Running on local machine
Parent Run ID: AutoML_de22720f-3457-4cb4-a8c8-29529b1b7325

Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing f

In [48]:
best_run, fitted_model = run.get_output()

In [50]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [51]:
model_name = best_run.properties['model_name']

script_file_name = 'inference/score.py'

best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'inference/score.py')

In [54]:
description = 'AutoML Iris model'
model = run.register_model(model_name = model_name, description = description, tags = None)

print("Model ID", run.model_id)

Model ID AutoMLde22720f330


In [60]:
## Use the below to prevent the 'No usable environment found' error

from azureml.core.environment import Environment
from azureml.automl.core.shared import constants
best_run.download_file(constants.CONDA_ENV_FILE_PATH, 'myenv.yml')
myenv = Environment.from_conda_specification(name="myenv", file_path="myenv.yml")

In [62]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import Webservice
from azureml.core.model import Model
from azureml.core.environment import Environment

inference_config = InferenceConfig(entry_script=script_file_name, environment=myenv)

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 1, 
                                               description = 'Iris classification service')

aci_service_name = 'automl-sample-bankmarketing-all'
print(aci_service_name)
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

automl-sample-bankmarketing-all
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running.........................................................................................................................................................................................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


In [61]:
#aci_service.delete()

In [65]:
X_test

,SepalLength,SepalWidth,PetalLength,PetalWidth,Class
73,6.10,2.80,4.70,1.20,Iris-versicolor
18,5.70,3.80,1.70,0.30,Iris-setosa
118,7.70,2.60,6.90,2.30,Iris-virginica
78,6.00,2.90,4.50,1.50,Iris-versicolor
76,6.80,2.80,4.80,1.40,Iris-versicolor
31,5.40,3.40,1.50,0.40,Iris-setosa
64,5.60,2.90,3.60,1.30,Iris-versicolor
141,6.90,3.10,5.10,2.30,Iris-virginica
68,6.20,2.20,4.50,1.50,Iris-versicolor
82,5.80,2.70,3.90,1.20,Iris-versicolor
